Bagging和随机森林

1.Bagging集成原理

    目标：把下面的圆圈和方块进行分类
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/Bagging集成原理图解-0.png)

    实现过程

    （1）采样不同数据集
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/Bagging集成原理图解-1.png)

    （2）训练分类器
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/Bagging集成原理图解-2.png)

    （3）平均投票，获取最终结果
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/Bagging集成原理图解-3.png)

    （4）主要实现过程小结
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/Bagging集成原理图解-4.png)

2.随机森林构造过程

    在机器学习中，随机森林是一个包含多个决策树的分类器，并且其输出的类别是由个别树输出
    的类别的众数而定。

    随机森林 = Bagging + 决策树
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/随机森林构造过程图解.png)

    例如，如果你训练了5棵树，其中有4个树的结果是True，1个树的结果是False，那么最终投票就是True
    随机森林构造过程中的关键步骤（M表示特征数目）：
        （1）一次随机选出一个样本，有放回抽样，重复N次（有可能出现重复的样本）
        （2）随机去选出m个特征，m<<M，建立决策树

    思考：
        （1）为什么要随机抽样训练集？
            如果不进行随机抽样，每棵树的训练集都是一样的，那么最终训练出的树分类结果
            也是完全一样的。
        （2）为什么要有放回抽样？
            如果不是有放回的抽样，那么每棵树的训练样本都是不同的，都是没有交集的，这
            样每棵树都是“有偏的”，都是绝对“片面的”（当然这么说可能不对），也就是说
            每棵树训练出来都是有很大的差异的；而随机森林最后分类取决于多棵树（弱分类
            器）的投票表决。

3.包外估计（Out-of-Bag Estimate）

    在随机森林构造的过程中，如果进行有放回的抽样，我们会发现总有一部分样本我们是选不到的。
    这部分数据，占整体数据的比重有多大？
    这部分数据有什么用？

        3.1.包外估计定义
        随机森林的Bagging过程，对于每一颗训练出的决策树gt，与数据集D有如下关系：
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/包外估计定义.png)

        对于星号的部分，即是没有选择到的数据，称之为Out-of-Bag（OOB）数据，当数据
        足够多的时候，对于任意一组数据（xn，yn）是包外数据的概率：
![jupyter](../Sources/Pictures/Ensemble_Learning_Algorithms/包外数据概率计算.png)

        由于基分类器是构建在训练样本的自助抽样集上，只有约63.2%原样本集出现在其中，
        而剩余36.8%的数据作为包外数据，可以用于基分类器的验证集。
        经验证：包外数据是对集成分类器泛化误差的无偏估计
        在随机森林算法中数据属性的重要性、分类器集强度和分类器间相关性计算都依赖于包外数据。
[什么是无偏估计](Expand/拓展1：无偏估计.ipynb)

        3.2.包外估计的用途
            （1）当基学习器是决策树时，可使用包外样本来辅助剪枝，或用于估计决策树中
            各结点的后验概率以辅助对零训练样本结点的处理。
            （2）当基学习器是神经网络时，可使用包外样本来辅助早期停止以减小过拟合。

4.随机森林API介绍

    sklearn.ensemble.RandomForestClassifier(
        n_estimators=10,criterion="gini",
        max_depth=None,Bootstrap=True,
        random_state=None,mini_samples_split=2
    )

    参数详解：
    n_estimator：类型：integer
                 可选，默认为10
                 森林里的数目数量：120，200，300，500，800，1200
    Criterion：类型：string
               可选，默认为gini
               作用：分割特征的测量方法
    max_depth:类型：integer或者None
              可选，默认为None
              树的最大深度5，8，15，25，30
    max_features:类型：string，每个决策树的最大特征数量
                 可选，默认为auto
                 if “auto”,then:   max_features=sqrt(n_features)
                 if "sqrt",then:   max_features=sqrt(n_features)(same as "auto")
                 if "log2",then:   max_features=log2(n_features)
                 if None,then:     max_features=n_features
    bootstrap:类型：boolean
              可选，默认为True
              作用：是否在构建树时使用有放回抽样
    min_samples_split:内部节点再划分所需最少样本数
            可选，默认值为2
            作用：这个值限制了子树继续划分的条件，
                 如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分
                 如果样本量不大，不需要管这个值。如果样本量数级非常大，则推荐增大这个值
    min_samples_leaf:叶子节点的最小样本数
            可选，默认值为1
            作用：这个值限制了叶子节点最少的样本数
                 如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝
            注意：叶子是决策树的末端节点，较小的叶子使得模型更容易波卓训练数据中的噪声
                一般来说，偏向于将最小叶子节点数目设置为大于50
    min_impurity_spllit:节点划分最小不纯度
            作用：这个值限制了决策树的增长，如果某节点的不纯度（即基尼系数，均方差）
                 小于这值，则该节点不再生成子节点，即为叶子节点。
            注意：一般不推荐改动默认值1e-7
    超参数：
        n_estimator,max_depth,min_samples_split,min_samples_leaf
    
    最重要的参数：
        最大特征数：max_features
        最大深度：max_depth
        内部节点再划分所需要最小样本数：min_samples_split
        叶子节点最小样本数：min_samples_leaf

    注意：随机森林的建立过程
         树的深度，数的个数等需要进行超参数调优

5.Bagging集成优点

    Bagging + 决策树/线性回归/逻辑回归/深度学习…… = Bagging集成学习方法
    
    经过上面方式组成的集成学习方法：
        （1）均可在原有算法上提高约2%左右的泛化正确率
        （2）简单、方便、通用

6.小结

    1、Bagging集成过程【知道】
        （1）采样--从所有样本里，采样一部分
        （2）学习--训练弱学习器
        （3）集成--使用平均投票
    2、随机森林介绍【知道】
        （1）随机森林定义：随机森林=Bagging+决策树
        （2）流程：
            （2.1）随机选取m条数据
            （2.2）随机选取k个特征
            （2.3）训练决策树
            （2.4）重复1~3
            （2.5）对上面的若决策树进行平均投票
        （3）注意：
            （3.1）随机选取样本呢，且是有放回的抽样
            （3.2）选取特征的时候，选择m << M
            （3.3）M是所有的特征数
        （4）包外估计
            如果进行有放回的对数据集抽样，会发现，总有一部分样本选不到
        （5）API
            sklearn.ensemble.RandomForestClassifier()
    3、Bagging + 决策树/线性回归/逻辑回归/深度学习…… = Bagging集成学习方法
    4、Bagging的优点【了解】
        （1）均可在原有算法上提高约2%左右的泛化正确率
        （2）简单、方便、通用